In [3]:
%load_ext autoreload
%autoreload 1
%aimport src
%aimport src.utils
%aimport src.SdmIsmAnalysis
%aimport src.SimpleSynthesis

# display last assigned variable automatically
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

%matplotlib inline
from matplotlib.pyplot import *

from src.utils import mygrid

import seaborn as sns
import polars as pl
import numpy as np
import scipy
from pathlib import Path
import os

from src.SimpleSynthesis import SimpleSynthesis

import pyfar as pf
import sofar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
rooms = ["HL05W", "HL06W"]

listeners = [
    "0.0X_0.0Y",
    "3.0X_2.0Y",
    "1.75X_-2.0Y",
    "-1.75X_-3.0Y",  # lower right corner in floor plan
    "-1.25X_0.0Y",
]

sources = range(5)

range(0, 5)

In [4]:
main_path = Path("../data/external/zenodo.10450779")

WindowsPath('../data/external/zenodo.10450779')

In [31]:
arr = []
for room in rooms:
    for listener in listeners:
        filepath = main_path / room / f"{room}_{listener}.sofa"
        print(filepath)
        sofa = sofar.read_sofa_as_netcdf(filepath)
        starts = np.abs(sofa.Data_IR[:, -1, 999])
        maxs = np.max(np.abs(sofa.Data_IR[:, -1, :]), axis=-1)
        for source in range(5):
            arr.append({
                "room": room,
                "listener": listener,
                "start_level": 20*np.log10(starts/maxs)[source],
                "prev_level": 20*np.log10(np.abs(sofa.Data_IR[:, -1, 998])/maxs)[source],
                "post_level": 20*np.log10(np.abs(sofa.Data_IR[:, -1, 1000])/maxs)[source],
                "source": source,
                "naive_onset": np.argmax(np.abs(sofa.Data_IR[source, -1, :]) > 0.1*maxs[source])
            })

..\data\external\zenodo.10450779\HL05W\HL05W_0.0X_0.0Y.sofa
..\data\external\zenodo.10450779\HL05W\HL05W_3.0X_2.0Y.sofa
..\data\external\zenodo.10450779\HL05W\HL05W_1.75X_-2.0Y.sofa
..\data\external\zenodo.10450779\HL05W\HL05W_-1.75X_-3.0Y.sofa
..\data\external\zenodo.10450779\HL05W\HL05W_-1.25X_0.0Y.sofa
..\data\external\zenodo.10450779\HL06W\HL06W_0.0X_0.0Y.sofa
..\data\external\zenodo.10450779\HL06W\HL06W_3.0X_2.0Y.sofa
..\data\external\zenodo.10450779\HL06W\HL06W_1.75X_-2.0Y.sofa
..\data\external\zenodo.10450779\HL06W\HL06W_-1.75X_-3.0Y.sofa
..\data\external\zenodo.10450779\HL06W\HL06W_-1.25X_0.0Y.sofa


In [32]:
df = pl.DataFrame(arr)

room,listener,start_level,prev_level,post_level,source,naive_onset
str,str,f64,f64,f64,i64,i64
"""HL05W""","""0.0X_0.0Y""",-18.162746,-33.432701,-16.839726,0,999
"""HL05W""","""0.0X_0.0Y""",-19.730728,-21.540117,-18.083269,1,999
"""HL05W""","""0.0X_0.0Y""",-14.008221,-34.838652,-1.578823,2,999
"""HL05W""","""0.0X_0.0Y""",-19.860269,-20.489427,-18.652082,3,999
"""HL05W""","""0.0X_0.0Y""",-18.864267,-46.626039,-24.747879,4,999
…,…,…,…,…,…,…
"""HL06W""","""-1.25X_0.0Y""",-19.261009,-21.138386,-17.832395,0,999
"""HL06W""","""-1.25X_0.0Y""",-17.167181,-21.218368,-14.199642,1,999
"""HL06W""","""-1.25X_0.0Y""",-12.354103,-23.097341,-3.288207,2,999


In [33]:
df.describe()

statistic,room,listener,start_level,prev_level,post_level,source,naive_onset
str,str,str,f64,f64,f64,f64,f64
"""count""","""50""","""50""",50.0,50.0,50.0,50.0,50.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,-14.157708,-26.375929,-14.112988,2.0,999.0
"""std""",null,null,6.446042,7.288226,9.364261,1.428571,0.0
"""min""","""HL05W""","""-1.25X_0.0Y""",-19.949325,-47.765506,-39.57227,0.0,999.0
"""25%""",null,null,-19.36311,-28.133139,-18.262627,1.0,999.0
"""50%""",null,null,-17.167181,-24.153493,-14.363874,2.0,999.0
"""75%""",null,null,-11.705689,-21.335636,-5.43016,3.0,999.0
"""max""","""HL06W""","""3.0X_2.0Y""",0.0,-20.15392,0.0,4.0,999.0


Since naive onset is always 999, I think we confirm how this was time aligned: The first sample above the threshold of -20 dB rel abs max at index 999 (MATLAB 1000).